In [1]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite

train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))


In [8]:
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

In [2]:
def word2features(sent,i):
    word=sent[i][0]
    postag=sent[i][1]
    features=[
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'postag[:2]=' + postag[:2],
    ]
    if i>0:
        word1=sent[i-1][0]
        postag1=sent[i-1][1]
        features=([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            '-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
    
    if i<len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            '+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
    return features
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]    

In [3]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

Wall time: 2.14 s


In [39]:
%%time
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

Wall time: 2.27 s


In [40]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [41]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [43]:
%%time
trainer.train('conll2002-esp.crfsuite')

Wall time: 12.5 s


In [44]:
!ls -lh ./conll2002-esp.crfsuite

-rw-r--r-- 1 jason 197609 409K Mar 10 19:09 ./conll2002-esp.crfsuite


In [45]:
trainer.logparser.last_iteration

{'active_features': 7183,
 'error_norm': 367.154101,
 'feature_norm': 107.05366,
 'linesearch_step': 1.0,
 'linesearch_trials': 1,
 'loss': 47845.715863,
 'num': 50,
 'scores': {},
 'time': 0.234}

In [47]:
len(trainer.logparser.iterations), trainer.logparser.iterations[-1]

(50,
 {'active_features': 7183,
  'error_norm': 367.154101,
  'feature_norm': 107.05366,
  'linesearch_step': 1.0,
  'linesearch_trials': 1,
  'loss': 47845.715863,
  'num': 50,
  'scores': {},
  'time': 0.234})

In [48]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002-esp.crfsuite')

In [49]:
example_sent = test_sents[0]
print(' '.join(sent2tokens(example_sent)), end='\n\n')

La Coruña , 23 may ( EFECOM ) .



In [50]:
print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

Predicted: B-LOC I-LOC O O O O B-ORG O O
Correct:   B-LOC I-LOC O O O O B-ORG O O


In [51]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [52]:
%%time
y_pred = [tagger.tag(xseq) for xseq in X_test]

Wall time: 399 ms


In [53]:
print(bio_classification_report(y_test, y_pred))

             precision    recall  f1-score   support

      B-LOC       0.72      0.32      0.45      1084
      I-LOC       0.61      0.53      0.57       325
     B-MISC       0.61      0.18      0.28       339
     I-MISC       0.68      0.40      0.50       557
      B-ORG       0.72      0.41      0.52      1400
      I-ORG       0.77      0.71      0.74      1104
      B-PER       0.85      0.67      0.75       735
      I-PER       0.81      0.84      0.83       634

avg / total       0.74      0.52      0.59      6178



In [4]:
dir(pycrfsuite)

['BaseTrainer',
 'CRFSUITE_VERSION',
 'CRFSuiteError',
 'ItemSequence',
 'Tagger',
 'Trainer',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_dumpparser',
 '_logparser',
 '_pycrfsuite',
 'absolute_import',
 'contextlib',
 'os',
 'sys',
 'tempfile']

In [6]:
help(pycrfsuite.Trainer)

Help on class Trainer in module pycrfsuite._pycrfsuite:

class Trainer(BaseTrainer)
 |  The trainer class.
 |  
 |  This class maintains a data set for training, and provides an interface
 |  to various training algorithms.
 |  
 |  Parameters
 |  ----------
 |  algorithm : {'lbfgs', 'l2sgd', 'ap', 'pa', 'arow'}
 |      The name of the training algorithm. See :meth:`Trainer.select`.
 |  
 |  params : dict, optional
 |      Training parameters. See :meth:`Trainer.set_params`
 |      and :meth:`Trainer.set`.
 |  
 |  verbose : boolean
 |      Whether to print debug messages during training. Default is True.
 |  
 |  Method resolution order:
 |      Trainer
 |      BaseTrainer
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  message(self, message)
 |      Trainer.message(self, message)
 |  
 |  on_end(self, log)
 |      Trainer.on_end(self, log)
 |  
 |  on_featgen_end(self, log)
 |      Trainer.on_featgen_end(self, log)
 |  
 |  on_featgen_progress(self, log, percent)
 |  